# Hybrid Search Demo
## Create Graph Schema

In [1]:
import os
os.environ["TG_HOST"] = "http://127.0.0.1"
os.environ["TG_USERNAME"] = "tigergraph"
os.environ["TG_PASSWORD"] = "tigergraph"

In [2]:
graph_schema = {
    "graph_name": "KGRec",
    "nodes": {
        "User": {
            "primary_key": "id",
            "attributes": {
                "id": "INT",
            },
        },
        "Song": {
            "primary_key": "id",
            "attributes": {
                "id": "INT",
                "description": "STRING",
            },
            "vector_attributes": {"emb_1": 1536},
        },
    },
    "edges": {
        "downloaded": {
            "is_directed_edge": False,
            "from_node_type": "User",
            "to_node_type": "Song",
        },
        "similar_to": {
            "is_directed_edge": False,
            "from_node_type": "Song",
            "to_node_type": "Song",
            "attributes": {
                "score": "DOUBLE",
            },
        },
    },
}

In [3]:
from tigergraphx import Graph
G = Graph(graph_schema)

2025-03-11 22:01:27,308 - tigergraphx.core.managers.schema_manager - INFO - Creating schema for graph: KGRec...
2025-03-11 22:03:05,515 - tigergraphx.core.managers.schema_manager - INFO - Graph schema created successfully.
2025-03-11 22:03:05,518 - tigergraphx.core.managers.schema_manager - INFO - Adding vector attribute(s) for graph: KGRec...
2025-03-11 22:03:58,557 - tigergraphx.core.managers.schema_manager - INFO - Vector attribute(s) added successfully.


## Load Data

In [4]:
loading_job_config = {
    "loading_job_name": "loading_job",
    "files": [
        {
            "file_alias": "f_song",
            "file_path": "/home/tigergraph/data/hybrid-search-demo/demo/data/song_embeddings.csv",
            "csv_parsing_options": {
                "separator": ",",
                "header": True,
            },
            "node_mappings": [
                {
                    "target_name": "Song",
                    "attribute_column_mappings": {
                        "id": "item_id",
                        "description": "description",
                        "emb_1": 'SPLIT($"embedding", " ")',
                    },
                }
            ],
        },
        {
            "file_alias": "f_downloads",
            "file_path": "/home/tigergraph/data/hybrid-search-demo/demo/data/implicit_lf_dataset.csv",
            "csv_parsing_options": {
                "separator": "\t",
                "header": False,
            },
            "node_mappings": [
                {
                    "target_name": "User",
                    "attribute_column_mappings": {
                        "id": 0,
                    },
                },
                {
                    "target_name": "Song",
                    "attribute_column_mappings": {
                        "id": 1,
                    },
                }
            ],
            "edge_mappings": [
                {
                    "target_name": "downloaded",
                    "source_node_column": 0,
                    "target_node_column": 1,
                }
            ],
        },
        {
            "file_alias": "f_similar_to",
            "file_path": "/home/tigergraph/data/hybrid-search-demo/demo/data/similar_songs.csv",
            "csv_parsing_options": {
                "separator": ",",
                "header": True,
            },
            "edge_mappings": [
                {
                    "target_name": "similar_to",
                    "source_node_column": "song_id_1",
                    "target_node_column": "song_id_2",
                    "attribute_column_mappings": {
                        "score": "similarity_score",
                    },
                }
            ],
        },
    ],
}

In [5]:
G.load_data(loading_job_config)

2025-03-11 22:04:12,900 - tigergraphx.core.managers.data_manager - INFO - Initiating data load for job: loading_job...
2025-03-11 22:04:20,255 - tigergraphx.core.managers.data_manager - INFO - Data load completed successfully.


---